In [1]:
%load_ext autoreload
from mps.state import *

# TIME EVOLUTION

In [2]:
# file: mps/evolution.py

import numpy as np
import scipy.linalg
from mps.state import _truncate_vector

Some of the commonly used operators:

In [3]:
# file: mps/evolution.py

σz = np.diag([1.0,-1.0])
i2 = np.identity(2)
σx = np.array([[0, 1], [1, 0]])
σy = -1j * σz @ σx

def creation_op(d):
    # Returns d dimensional cration operator
    return np.diag(np.sqrt(np.arange(1,d)),-1).astype(complex)

def annihilation_op(d):
    # Returns d dimensional cration operator
    return np.diag(np.sqrt(np.arange(1,d)),1).astype(complex)



## Nearest Neighbor Hamiltonians

Below is the interface we use for nearest neighbor interaction Hamiltonians. It is initially empty.

In [4]:
# file: mps/evolution.py

class NNHamiltonian(object):
    
    def __init__(self, size):
        #
        # Create a nearest-neighbor interaction Hamiltonian
        # of a given size, initially empty.
        #
        self.size = size
        
    def dimension(self, ndx, t=0.0):
        #
        # Return the dimension of the local Hilbert space
        #
        return 0
    
    def interaction_term(self, ndx, t=0.0):
        #
        # Return the interaction between sites (ndx,ndx+1)
        #
        return 0

Generally, we can have different local terms and different interactions on every site
$$
H = \sum_i \left[O_i + \sum_n L^{(n)}_i \otimes R^{(n)}_{i+1}\right]_\text{site i} 
=  \sum_{i=0}^{N-2} h_{i,i+1}. 
$$
where
$$
h_{i,i+1} = \sum_n L^{(n)}_i \otimes R^{(n)}_{i+1} +
\begin{cases}
O_i + \frac{1}{2} O_{i+1}, \text{ if  } i = 0 \\
\frac{1}{2} O_i + O_{i+1}, \text{ if  } i = N-2 \\
\frac{1}{2} O_i + \frac{1}{2} O_{i+1}, \text{ else  } 
\end{cases}
$$
and $N>2$.

The function below computes the interaction terms $h_{i,i+1}$:

In [5]:
# file: mps/evolution.py

def _compute_interaction_term(H, ndx, t=0.0):
    """Computes the interaction term between site ndx and ndx+1, including the local terms 
    for the two sites
    
    Arguments:
    H = NNHamiltonian
    ndx = site index
    """
    if isinstance(H.local_terms[ndx], np.ndarray ):            
        if ndx == 0:
            H.interactions[ndx] +=  np.kron(H.local_terms[ndx],
                                               np.eye(H.dimension(ndx+1)))
        else:
            H.interactions[ndx] +=  0.5 * np.kron(H.local_terms[ndx],
                                                     np.eye(H.dimension(ndx+1)))
    if isinstance(H.local_terms[ndx+1], np.ndarray ):            
        if ndx == H.size-2:
            H.interactions[ndx] +=  np.kron(np.eye(H.dimension(ndx)),
                                               H.local_terms[ndx+1])
        else:
            H.interactions[ndx] +=  0.5 * np.kron(np.eye(H.dimension(ndx)),
                                                     H.local_terms[ndx+1])

    return H.interactions[ndx]

In [6]:
# file: mps/evolution.py

class ConstantNNHamiltonian(NNHamiltonian):
    
    def __init__(self, size):
        #
        # Create a nearest-neighbor interaction Hamiltonian with fixed
        # local terms and interactions.
        #
        #  - local_term: operators acting on each site (can be different for each site)
        #  - int_left, int_right: list of L and R operators (can be different for each site)
        #
        self.size = size
        self.local_terms = [0] * size
        self.int_left = [[]] * (size-1)
        self.int_right = [[]] * (size-1)
        self.interactions = [0] * (size-1)

    def set_local_term(self, ndx, operator):
        #
        # Set the local term acting on the given site
        #
        self.local_terms[ndx] = operator

    def add_interaction_term(self, ndx, L, R):
        #
        # Add an interaction term $L \otimes R$ acting on sites 'ndx' and 'ndx+1'
        #
        # Add to int_left, int_right
        #
        # Update the self.interactions[ndx] term
        self.int_left[ndx].append(L)
        self.int_right[ndx].append(R)
        self.interactions[ndx] += np.kron(L,R)
        
    def dimension(self, ndx, t=0.0):
        #
        # Return the dimension of the local Hilbert space
        #
        if ndx == self.size -1:
            return self.int_right[ndx-1][0].shape[0]
        else:
            return self.int_left[ndx][0].shape[0]
    
    #def interaction_term(self, ndx, t=0.0):
        #
        # Return the interaction between sites (ndx,ndx+1) including the corresponding local terms.
        #
        #return _compute_interaction_term(self, ndx, t=0.0)

A particular case would be a translationally invariant, constant Hamiltonian
$$H = \sum_i \left[O + \sum_n L^{(n)} \otimes R^{(n)}\right]_\text{site i}$$
which has the same local term $O$ on all sites, and the same interaction given by the product of $L^{(n)}$ left and $R^{(n)}$ right operators.

In [7]:
# file: mps/evolution.py

class TINNHamiltonian(ConstantNNHamiltonian):
    
    def __init__(self, size, local_term, intL, intR):
        #
        # Create a constant nearest-neighbor interaction Hamiltonian with fixed
        # local terms and interactions.
        #
        #  - local_term: operator acting on every site
        #  - int_left: list of L (applied to site ndx) operators
        #  - int_right: list of R (applied to site ndx + 1) operators
        #  - interaction: kronecker product of corresponding L and R pairs
        #
        self.size = size
        self.local_terms = [local_term] * size
        self.int_left = [[]] * (size-1)
        self.int_right = [[]] * (size-1)
        self.intL = intL
        self.intR = intR
        self.interactions = [0] * (size-1)
    
    def interaction_term(self, ndx, t=0.0):
        #
        # Return the interaction between sites (ndx,ndx+1)
        #
        if isinstance(self.interactions[ndx], np.ndarray):
            return self.interactions[ndx]
        
        else:
            for L,R in zip(self.intL, self.intR):
                self.add_interaction_term(ndx, L, R)
               
            return _compute_interaction_term(self, ndx, t=0.0)
        

## Suzuki-Trotter Decomposition

In Suzuki-Trotter decomposition, the Hamiltonians of the nearest neighbor couplings can be decomposed into two non-commuting parts, $H_{\text{odd}} $ and $ H_{\text{even}} $, so that all additive 2-site operators in each part commute with each other.

Let us consider a simple example of tight binding model with on-site potential and decompose the Hamiltonian into 2-site terms, so that $H=\sum_i h_{i,i+1}$. 
\begin{equation}
h_{i,i+1} = \left(\frac{\omega}{2}  a_i^\dagger a_i \right) + \left(\frac{\omega}{2}  a_{i+1}^\dagger a_{i+1} \right) - \left( t a_{i}^\dagger a_{i+1} + \text{h.c.} \right).
\end{equation}
Since $[h_{i,i+1},h_{i+2,i+3}] = 0$, we can group these terms for even and odd $i$, so that $H = H_{\text{odd}} + H_{\text{even}} $. 

Note that the local term $ a_i^\dagger a_i$ appears only in one the groups for $i=1$ and $i=N$. Therefore we need to add two on-site terms $h_1 = \left(\frac{\omega}{2}  a_1^\dagger a_1 \right) $ and $h_N = \left(\frac{\omega}{2}  a_N^\dagger a_N \right) $, separately. So that $H_{\text{even}} \rightarrow H_{\text{evem}} + h_1$, and $h_N$ is included similarly depending on whether $N$ is even or odd.

And for the first order Suzuki-Trotter decomposition, the evolution operator becomes
\begin{equation}
e^{-i \hat{H} \Delta t} = e^{-i \hat{H}_{\text{odd}} \Delta t}  e^{-i \hat{H}_{\text{even}} \Delta t} + O(\Delta t^2).
\end{equation}

The class Trotter_unitaries creates a list of Trotter unitarities depending on evenodd value. 

If evenodd = 0:
$$ U_{\text{odds}} = [U_{1,2}, U_{3,4}, \dots ], $$
and if evenodd = 1:
$$ U_{\text{evens}} = [U_1, U_{2,3}, U_{4,5}, \dots ], $$
where $U_i = e^{-i h_{i} \Delta t}$ and $U_{i,i+1} = e^{-i h_{i,i+1} \Delta t}$. 

Note that the last element of  $U_{\text{odds/evens}}$ is $U_N$ if $N$ is odd/even, and $U_{N-1,N}$ otherwise.

In [8]:
# file: mps/evolution.py

class Trotter_unitaries(object):
    """"Create Trotter unitarities from a nearest-neighbor interaction Hamiltonian.
        
    Attributes:
    H = NNHamiltonian
    δt = Time step
    """

    def __init__(self, H, δt):
        self.H = H
        self.δt = δt
                
    def twosite_unitary(self, start):
        """Creates twp-site exponentials from interaction H terms"""
        U = scipy.linalg.expm(-1j * self.δt * self.H.interaction_term(start))
        U = U.reshape(self.H.dimension(start),self.H.dimension(start+1),
                      self.H.dimension(start),self.H.dimension(start+1))
        return U
    
    

We apply each $U_{i,i+1} = e^{-i h_{i,i+1} \Delta t}$ to two neighbouring tensors, $A_i$ and $A_{i+1}$ simultaneously, as shown below.

<img src="fig_pdf/apply_mpo_to2site.svg" style="max-width: 90%; width: 35em">

The resulting tensor $B$ is a two-site tensor. We split this tensor into two one-site tensors using left_orth_2site and right_orth_2site functions, which are defined in [this notebook](File%201c%20-%20Canonical%20form.ipynb). 

In [9]:
# file: mps/evolution.py

def apply_2siteTrotter(U, ψ, start):
    return np.einsum('ijk,klm,prjl -> iprm', ψ[start],
                     ψ[start+1], U)


In [10]:
# file: mps/evolution.py

def TEBD_sweep(H, ψ, δt, dr, evenodd, tol=0):
    #
    # Apply a TEBD sweep by evolving with the pairwise Trotter Hamiltonian
    # starting from left/rightmost site and moving on the 'direction' (>0 right,
    # <0 left) by pairs of sites.
    #
    # - H: NNHamiltonian
    # - ψ: Initial state in CanonicalMPS form (modified destructively)
    # - δt: Time step
    # - evenodd: 0, 1 depending on Trotter step
    # - direction: where to move
    #
    Trotter = Trotter_unitaries(H, δt)        
    def update_two_site(start, nextsite, dr):
        # Apply combined local and interaction exponential and move
        ψ.center = ψ.center + dr
        if start == 0:
            dr = +1
        elif start == (ψ.size-2):
            dr = -1
        AA = apply_2siteTrotter(Trotter.twosite_unitary(start) , ψ, start)
        ψ.update_canonical_2site(AA, start, nextsite, dr, tolerance=tol)

    #
    # Loop over ψ, updating pairs of sites acting with the unitary operator
    # made of the interaction and 0.5 times the local terms
    #
    if dr < 0:
        for j in range(ψ.size-2, -1, -2):
            print(ψ.center)
            update_two_site(j, j+1, dr)
    else:
        for j in range(0, ψ.size-1, +2):
            print(ψ.center)
            update_two_site(j, j+1, dr)        
            
    return ψ



## Error in Suzuki-Trotter decomposition

In the first order Suzuki-Trotter decomposition, evolution operator becomes
\begin{equation}
e^{-i \hat{H} \Delta t} = e^{-i \hat{H}_{\text{odd}} \Delta t}  e^{-i \hat{H}_{\text{even}} \Delta t} + O(\Delta t^2).
\end{equation}
Note that after $T/\Delta t$ time steps, the accumulated error is in the order of $\Delta t$.
Higher order Suzuki-Trotter decompositions can be used to reduce error.




# Tests

In [25]:
# file: mps/test/test_TEBD.py
import unittest
import mps.state
import mps.tools
from mps.evolution import *
from mps.test.tools import *
import scipy.sparse as sp

N = 7
t = 1
ω = 0.5
        
def random_wavefunction(n):
    ψ = np.random.rand(n) - 0.5
    return ψ / np.linalg.norm(ψ)

class TestTEBD_sweep(unittest.TestCase):
    
    def test_orthonormalization(self):
        #
        # We verify that our two-site orthonormalization procedure, 
        # does not change the state
        #
        δt = 0.1

        def ok(Ψ):
            H = TINNHamiltonian(Ψ.size, 0*σx, [mps.tools.random_Pauli()], [mps.tools.random_Pauli()])
            Trotter = Trotter_unitaries(H, δt)
            for start in range(Ψ.size-2):
                AA = apply_2siteTrotter(Trotter.twosite_unitary(start) , 
                                                      Ψ, start)
                A, AC = mps.state.left_orth_2site(AA, DEFAULT_TOLERANCE)
                AA_orth = np.einsum("ijk,klm -> ijlm", A, AC)
                self.assertTrue(similar(AA,AA_orth))    
                
                AA = apply_2siteTrotter(Trotter.twosite_unitary(start) , 
                                                      Ψ, start)
                A, AC = mps.state.right_orth_2site(AA, DEFAULT_TOLERANCE)
                AA_orth = np.einsum("ijk,klm -> ijlm", AC, A)
                self.assertTrue(similar(AA,AA_orth))
            
            
        test_over_random_mps(ok)
        
    def test_right_sweep(self):
        #
        # We verify the truncation procedure does not change the resulting state. 
        # We evolve the mps only through one sweep to eliminate the error due to 
        # non-commuting terms in the Hamiltonian. We compare the TEBD results with 
        # exact diagonalization results.
        #
        # Note that there is a phase difference between the two wavefunctions.
        # However absolute values of the corresponding coefficients are equal 
        # as the test verifies.
        #                
        dt = 1
        ψwave = random_wavefunction(N)
        ψmps = CanonicalMPS(mps.state.wavepacket(ψwave), center=0)
        # We use the tight-binding Hamiltonian
                
        H=TINNHamiltonian(N, ω*creation_op(2)@ annihilation_op(2), 
                            [t * annihilation_op(2) , t * creation_op(2)], 
                            [creation_op(2), annihilation_op(2)])
        ψmps_final = TEBD_sweep(H, ψmps, dt, 1, 0, tol=DEFAULT_TOLERANCE)
        Hmat = sp.diags([[t,0]*(N//2), ω, [t,0]*(N//2)],
                  offsets=[-1,0,+1],
                  shape=(N,N),
                  dtype=np.complex128)
        ψwave_final = sp.linalg.expm_multiply(-1j * dt * Hmat, ψwave)
        print(abs(mps.state.wavepacket(ψwave_final).tovector()))
        print(abs(ψmps_final.tovector()))
        print(np.vdot(abs(mps.state.wavepacket(ψwave_final).tovector()), abs(ψmps_final.tovector())))
        self.assertTrue(similar(abs(mps.state.wavepacket(ψwave_final).tovector()), 
                                abs(ψmps_final.tovector())))
     
    
    def test_left_sweep(self):
        #
        # We verify the truncation procedure does not change the resulting state. 
        # We evolve the mps only through one sweep to eliminate the error due to 
        # non-commuting terms in the Hamiltonian. We compare the TEBD results with 
        # exact diagonalization results.
        #
        # Note that there is a phase difference between the two wavefunctions.
        # However absolute values of the corresponding coefficients are equal 
        # as the test verifies.
        #                
        dt = 1
        ψwave = random_wavefunction(N)
        ψmps = CanonicalMPS(mps.state.wavepacket(ψwave), center=N-1)
        # We use the tight-binding Hamiltonian
                
        H=TINNHamiltonian(N, ω*creation_op(2)@ annihilation_op(2), 
                            [t * annihilation_op(2) , t * creation_op(2)], 
                            [creation_op(2), annihilation_op(2)])
        
  
        ψmps_final = TEBD_sweep(H, ψmps, dt, -1, 0, tol=DEFAULT_TOLERANCE)
        Hmat = sp.diags([[0,t]*(N//2), ω, [0,t]*(N//2)],
                  offsets=[-1,0,+1],
                  shape=(N,N),
                  dtype=np.complex128)
        ψwave_final = sp.linalg.expm_multiply(-1j * dt * Hmat, ψwave)
        print(abs(mps.state.wavepacket(ψwave_final).tovector()))
        print(abs(ψmps_final.tovector()))
        print(np.vdot(abs(mps.state.wavepacket(ψwave_final).tovector()), abs(ψmps_final.tovector())))
        
        self.assertTrue(similar(abs(mps.state.wavepacket(ψwave_final).tovector()), 
                                abs(ψmps_final.tovector())))
     



In [26]:
%autoreload
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestTEBD_sweep'])
unittest.TextTestRunner(verbosity=2).run(suite1);

test_left_sweep (__main__.TestTEBD_sweep) ... FAIL
test_orthonormalization (__main__.TestTEBD_sweep) ... 

6
4
2
[ 0.          0.22619679  0.34182869  0.          0.46041405  0.          0.
  0.          0.44526872  0.          0.          0.          0.          0.
  0.          0.          0.31322022  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.4104937   0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.39386653  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.      

ok
test_right_sweep (__main__.TestTEBD_sweep) ... 

0
2
4
[ 0.          0.37539447  0.49063062  0.          0.38468262  0.          0.
  0.          0.4239889   0.          0.          0.          0.          0.
  0.          0.          0.30555248  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.2929899   0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.33377819  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.      

FAIL

FAIL: test_left_sweep (__main__.TestTEBD_sweep)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-25-d663c40736c6>", line 109, in test_left_sweep
    abs(ψmps_final.tovector())))
AssertionError: False is not true

FAIL: test_right_sweep (__main__.TestTEBD_sweep)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-25-d663c40736c6>", line 74, in test_right_sweep
    abs(ψmps_final.tovector())))
AssertionError: False is not true

----------------------------------------------------------------------
Ran 3 tests in 0.905s

FAILED (failures=2)


In [13]:
N = 4
t = 1
ω = 0
H=TINNHamiltonian(N, ω*creation_op(2)@ annihilation_op(2), 
                            [t * annihilation_op(2) , t * creation_op(2)], 
                            [creation_op(2), annihilation_op(2)])
        
   

In [14]:
H.interaction_term(0)

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [15]:
δt=1
Trotter = Trotter_unitaries(H, δt)

In [16]:
Trotter.twosite_unitary(2)[1,:,:,:]

array([[[ 0.00000000+0.j        ,  0.00000000-0.84147098j],
        [ 0.54030231+0.j        ,  0.00000000+0.j        ]],

       [[ 0.00000000+0.j        ,  0.00000000+0.j        ],
        [ 0.00000000+0.j        ,  1.00000000+0.j        ]]])

In [17]:
Hmat = sp.diags([[0,t]*(N//2), ω, [0,t]*(N//2)],
                  offsets=[-1,0,+1],
                  shape=(N,N),
                  dtype=np.complex128)

In [18]:
Hmat.shape

(4, 4)

In [19]:
Hmat.todense()

matrix([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
        [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [20]:
N = 4
Hmat = sp.diags([[0,t]*(N//2), ω, [0,t]*(N//2)],
                  offsets=[-1,0,+1],
                  shape=(N,N),
                  dtype=np.complex128)

Hmat.todense()

matrix([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
        [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]])